In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,2021-01-30T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1345,208,1553,41315,...,NaN,8.0,NaN,NaN,135387.0,0.0,1498199.0,253410.0,ITG,ITG2
7178,2021-01-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,648,95,743,8340,...,NaN,1.0,NaN,NaN,133853.0,93.0,2141135.0,81997.0,ITG,ITG1
7179,2021-01-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,346,55,401,5110,...,Si fa presente come un ricovero (in terapia in...,6.0,NaN,NaN,35757.0,0.0,591529.0,31629.0,ITI,ITI2
7180,2021-01-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,35,4,39,209,...,NaN,1.0,NaN,NaN,7781.0,0.0,68965.0,866.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,2021-01-30T17:00:00,19,Sicilia,1345,208,1553,41315,42868,-1399,846,...,NaN,NaN,135387,1751609,980665.0,8.0,135387.0,0.0,1498199.0,253410.0
7178,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,NaN,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0
7179,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,NaN,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0
7180,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,NaN,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-30', '2021-01-29')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,NaN,NaN,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0
1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,NaN,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0
2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,NaN,NaN,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0
3,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,NaN,NaN,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0
4,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,415,...,NaN,NaN,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0
5,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,255,...,NaN,NaN,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0
6,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,NaN,NaN,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0
7,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,NaN,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0
8,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,NaN,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0
9,2021-01-30T17:00:00,11,Marche,542,71,613,8547,9160,153,465,...,NaN,NaN,55125,685976,412859.0,3.0,55125.0,0.0,661459.0,24517.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-29T17:00:00,1,Piemonte,2163,150,2313,10224,12537,53,944,...,NaN,NaN,221832,2040494,1090338.0,13.0,220099.0,1733.0,1872204.0,168290.0
1,2021-01-29T17:00:00,2,Valle d'Aosta,36,4,40,213,253,-10,8,...,NaN,NaN,7780,69596,41328.0,0.0,7780.0,0.0,68858.0,738.0
2,2021-01-29T17:00:00,3,Lombardia,3490,379,3869,45645,49514,-325,1900,...,NaN,NaN,534784,5601457,2820413.0,28.0,533337.0,1447.0,5481950.0,119507.0
3,2021-01-29T17:00:00,5,Veneto,1509,240,1749,35216,36965,-1999,985,...,NaN,NaN,310894,4075490,1317694.0,11.0,309583.0,1311.0,3728265.0,347225.0
4,2021-01-29T17:00:00,6,Friuli Venezia Giulia,605,59,664,10707,11371,11,557,...,NaN,NaN,66723,1134427,412664.0,7.0,64445.0,2278.0,1101914.0,32513.0
5,2021-01-29T17:00:00,7,Liguria,591,64,655,3650,4305,59,338,...,NaN,NaN,69106,849066,373287.0,1.0,69106.0,0.0,813393.0,35673.0
6,2021-01-29T17:00:00,8,Emilia-Romagna,2172,209,2381,44964,47345,-194,1320,...,NaN,NaN,216065,3066158,1349723.0,14.0,216016.0,49.0,2963932.0,102226.0
7,2021-01-29T17:00:00,9,Toscana,659,100,759,8243,9002,328,594,...,NaN,NaN,133343,2207275,1181251.0,6.0,133264.0,79.0,2131187.0,76088.0
8,2021-01-29T17:00:00,10,Umbria,349,53,402,5075,5477,174,335,...,NaN,NaN,35529,616083,280987.0,4.0,35529.0,0.0,587593.0,28490.0
9,2021-01-29T17:00:00,11,Marche,540,73,613,8394,9007,-20,371,...,NaN,NaN,54660,678817,409342.0,5.0,54660.0,0.0,655742.0,23075.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,NaN,NaN,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0
1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,NaN,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0
2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,NaN,NaN,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0
3,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,0.0,0.0,67092,1018897,340192.0,3.0,61127.0,5965.0,931630.0,87267.0
4,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,NaN,NaN,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0
5,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,415,...,NaN,NaN,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0
6,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,255,...,NaN,NaN,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0
7,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,NaN,NaN,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0
8,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,NaN,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0
9,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,NaN,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-01-29T17:00:00,1,Piemonte,2163,150,2313,10224,12537,53,...,NaN,NaN,221832,2040494,1090338.0,13.0,220099.0,1733.0,1872204.0,168290.0
1,1,2021-01-29T17:00:00,2,Valle d'Aosta,36,4,40,213,253,-10,...,NaN,NaN,7780,69596,41328.0,0.0,7780.0,0.0,68858.0,738.0
2,2,2021-01-29T17:00:00,3,Lombardia,3490,379,3869,45645,49514,-325,...,NaN,NaN,534784,5601457,2820413.0,28.0,533337.0,1447.0,5481950.0,119507.0
3,19,2021-01-29T17:00:00,4,Trentino-Alto Adige,406,64,470,16763,17233,530,...,0.0,0.0,66222,1007944,339170.0,1.0,60824.0,5398.0,927180.0,80764.0
4,3,2021-01-29T17:00:00,5,Veneto,1509,240,1749,35216,36965,-1999,...,NaN,NaN,310894,4075490,1317694.0,11.0,309583.0,1311.0,3728265.0,347225.0
5,4,2021-01-29T17:00:00,6,Friuli Venezia Giulia,605,59,664,10707,11371,11,...,NaN,NaN,66723,1134427,412664.0,7.0,64445.0,2278.0,1101914.0,32513.0
6,5,2021-01-29T17:00:00,7,Liguria,591,64,655,3650,4305,59,...,NaN,NaN,69106,849066,373287.0,1.0,69106.0,0.0,813393.0,35673.0
7,6,2021-01-29T17:00:00,8,Emilia-Romagna,2172,209,2381,44964,47345,-194,...,NaN,NaN,216065,3066158,1349723.0,14.0,216016.0,49.0,2963932.0,102226.0
8,7,2021-01-29T17:00:00,9,Toscana,659,100,759,8243,9002,328,...,NaN,NaN,133343,2207275,1181251.0,6.0,133264.0,79.0,2131187.0,76088.0
9,8,2021-01-29T17:00:00,10,Umbria,349,53,402,5075,5477,174,...,NaN,NaN,35529,616083,280987.0,4.0,35529.0,0.0,587593.0,28490.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,NaN,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0,4341375
1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0,125501
2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,NaN,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0,10103969
3,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,0.0,67092,1018897,340192.0,3.0,61127.0,5965.0,931630.0,87267.0,1074819
4,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,NaN,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0,4907704
5,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,415,...,NaN,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0,1211357
6,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,255,...,NaN,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0,1543127
7,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,NaN,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0,4467118
8,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0,3722729
9,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0,4341375,575
1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0,125501,20
2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0,10103969,1036
3,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,67092,1018897,340192.0,3.0,61127.0,5965.0,931630.0,87267.0,1074819,106
4,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0,4907704,1016
5,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,415,...,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0,1211357,180
6,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,255,...,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0,1543127,209
7,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0,4467118,634
8,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0,3722729,523
9,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,36.39,665,-68,39462,9230.0,58,1147,4.64,0.01813,5.31
1,2021-01-30T17:00:00,15,Campania,1435,92,1527,60129,61656,14,1366,...,18.22,7,191,17345,9969.0,22,1330,7.88,0.02361,3.82
2,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,32.02,-932,-6,24020,4566.0,27,2271,5.47,0.02941,4.87
3,2021-01-30T17:00:00,12,Lazio,2407,282,2689,61106,63795,-442,1138,...,33.29,-404,-22,30386,19597.0,27,1553,3.75,0.01940,3.49
4,2021-01-30T17:00:00,16,Puglia,1389,163,1552,50605,52157,-121,871,...,44.17,-104,-198,9690,4083.0,29,963,8.99,0.02173,3.04
5,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,63.21,577,-27,10953,1022.0,9,298,7.94,0.08094,6.24
6,2021-01-30T17:00:00,19,Sicilia,1345,208,1553,41315,42868,-1399,846,...,35.37,-1368,-98,25251,5448.0,35,2210,3.35,0.01703,2.72
7,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,21.85,-2199,-193,57107,3053.0,60,3020,1.39,0.01614,6.35
8,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,26.09,-119,-217,23052,5374.0,59,823,3.15,0.01675,5.13
9,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,18.16,97,9,15857,6994.0,22,500,3.80,0.01620,3.60


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,67,39,106,63.21
1,Marche,71,72,143,49.65
2,Umbria,55,62,117,47.01
3,Puglia,163,206,369,44.17
4,Lombardia,377,659,1036,36.39
5,Sicilia,208,380,588,35.37
6,Lazio,282,565,847,33.29
7,Emilia-Romagna,203,431,634,32.02
8,Friuli Venezia Giulia,57,123,180,31.67
9,Liguria,62,147,209,29.67


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,2021-01-28T17:00:00,22,P.A. Trento,191,34,225,1938,2163,95,207,...,NaN,NaN,26855,519585,160847.0,2.0,25302.0,1553.0,505405.0,14180.0
680,2021-01-29T17:00:00,21,P.A. Bolzano,219,30,249,14613,14862,322,590,...,NaN,NaN,39060,484791,178089.0,0.0,35447.0,3613.0,419450.0,65341.0
681,2021-01-29T17:00:00,22,P.A. Trento,187,34,221,2150,2371,208,307,...,NaN,NaN,27162,523153,161081.0,1.0,25377.0,1785.0,507730.0,15423.0
682,2021-01-30T17:00:00,21,P.A. Bolzano,216,32,248,15083,15331,469,640,...,NaN,NaN,39700,492413,178834.0,2.0,35711.0,3989.0,422081.0,70332.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2021-01-26T17:00:00,4,Trentino-Alto Adige,441,66,507,15518,16025,382,716,...,15.0,11093.0,748.0,398.0,13.02,3.16,6.45,95.72,-4.35,2.44
338,2021-01-27T17:00:00,4,Trentino-Alto Adige,418,67,485,15827,16312,287,790,...,4.0,11806.0,1053.0,309.0,13.81,2.97,6.69,75.02,1.52,1.79
339,2021-01-28T17:00:00,4,Trentino-Alto Adige,407,64,471,16232,16703,391,769,...,4.0,11509.0,1022.0,405.0,13.59,2.82,6.68,75.24,-4.48,2.40
340,2021-01-29T17:00:00,4,Trentino-Alto Adige,406,64,470,16763,17233,530,897,...,4.0,11105.0,858.0,531.0,13.62,2.73,8.08,104.55,0.00,3.17


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
